In [4]:
import pandas as pd
import numpy as np
from scipy.special import erf  # Added for erf function

def norm_cdf(x):
    return 0.5 * (1 + erf(x / np.sqrt(2)))

def mann_kendall(y, alpha=0.05):
    y = np.array(y)[~np.isnan(y)]
    n = len(y)
    if n < 2:
        return 'Insufficient Data', np.nan, np.nan, False
    # Calculate S
    s = 0
    for i in range(n-1):
        for j in range(i+1, n):
            s += np.sign(y[j] - y[i])
    # Count ties
    unique, counts = np.unique(y, return_counts=True)
    t = counts[counts > 1]
    tt = np.sum(t * (t - 1) * (2 * t + 5))
    # Variance
    var = (n * (n - 1) * (2 * n + 5) - tt) / 18
    if var == 0:
        z = 0
    else:
        z = (s - np.sign(s)) / np.sqrt(var) if s > 0 else (s + 1) / np.sqrt(var) if s < 0 else 0
    # p-value 2-tailed
    p = 2 * (1 - norm_cdf(np.abs(z))) if var > 0 else 1.0
    h = p < alpha
    trend = "Increasing" if z > 0 else "Decreasing" if z < 0 else "No Trend"
    return trend, p, z, h

def sens_slope(y):
    y = np.array(y)[~np.isnan(y)]
    n = len(y)
    if n < 2:
        return np.nan
    slopes = []
    for i in range(n-1):
        for j in range(i+1, n):
            slopes.append((y[j] - y[i]) / (j - i))
    return np.median(slopes)

# Load the monthly averages data from the file
monthly_file = r"D:\climate change\monthly_averages.xlsx"

# Read the specific sheets for rainfall, tmax, and tmin
rain_df = pd.read_excel(monthly_file, sheet_name='Rainfall_monthly', index_col=0)
tmax_df = pd.read_excel(monthly_file, sheet_name='Tmax_monthly', index_col=0)
tmin_df = pd.read_excel(monthly_file, sheet_name='Tmin_monthly', index_col=0)

# Drop 'Lon' and 'Lat' rows from each DataFrame
rain_df = rain_df.drop(['Lon', 'Lat'], errors='ignore')
tmax_df = tmax_df.drop(['Lon', 'Lat'], errors='ignore')
tmin_df = tmin_df.drop(['Lon', 'Lat'], errors='ignore')

# Compute mean temperature DataFrame
temp_df = (tmax_df + tmin_df) / 2

# Ensure the index is a DatetimeIndex
temp_df.index = pd.to_datetime(temp_df.index)
rain_df.index = pd.to_datetime(rain_df.index)

# Get data values
temp_data_values = temp_df.values
rain_data_values = rain_df.values

# Get station names
station_names = temp_df.columns

# Initialize a list to store Mann-Kendall test results
mk_results = []

# Perform Mann-Kendall test for each station
for i in range(temp_data_values.shape[1]):
    temperature = temp_data_values[:, i]
    precipitation = rain_data_values[:, i]
    
    # For temperature
    trend_temp, p_temp, z_temp, h_temp = mann_kendall(temperature)
    slope_temp = sens_slope(temperature)
    temp_result = {
        'Station': station_names[i],
        'Variable': 'Temperature',
        'Trend': trend_temp,
        'p-value': p_temp,
        'Z': z_temp,
        'Slope': slope_temp,
        'Significant': h_temp
    }
    
    # For precipitation
    trend_rain, p_rain, z_rain, h_rain = mann_kendall(precipitation)
    slope_rain = sens_slope(precipitation)
    rain_result = {
        'Station': station_names[i],
        'Variable': 'Precipitation',
        'Trend': trend_rain,
        'p-value': p_rain,
        'Z': z_rain,
        'Slope': slope_rain,
        'Significant': h_rain
    }
    
    mk_results.extend([temp_result, rain_result])

# Convert to DataFrame
mk_df = pd.DataFrame(mk_results)

# Save the Mann-Kendall test results to CSV
output_path = r"D:\climate change\mann_kendall_results.csv"
mk_df.to_csv(output_path, index=False)

print(f"Mann-Kendall test results saved to {output_path}")
print(mk_df.to_markdown(index=False))

Mann-Kendall test results saved to D:\climate change\mann_kendall_results.csv
| Station    | Variable      | Trend      |     p-value |          Z |        Slope | Significant   |
|:-----------|:--------------|:-----------|------------:|-----------:|-------------:|:--------------|
| Station-1  | Temperature   | Increasing | 0.0159483   |  2.4101    |  0.00203793  | True          |
| Station-1  | Precipitation | Decreasing | 0.00162358  | -3.15164   | -0.00414404  | True          |
| Station-2  | Temperature   | Increasing | 0.0316664   |  2.1486    |  0.00163493  | True          |
| Station-2  | Precipitation | Decreasing | 0.610653    | -0.509142  | -0.000223091 | False         |
| Station-3  | Temperature   | Increasing | 0.322466    |  0.989403  |  0.00115219  | False         |
| Station-3  | Precipitation | Increasing | 0.704057    |  0.37985   |  4.21676e-05 | False         |
| Station-4  | Temperature   | Increasing | 0.197472    |  1.28879   |  0.000586339 | False         |
| St